In [2]:
import urllib.request
from urllib.request import Request, urlopen

import requests
import json
import jsonpath
import jsonlines

import pandas as pd

In [ ]:
'''
# request url

#url = "https://api-samenmeten.rivm.nl/v1.0/Things"
#response = requests.get(url)
#html = response.text

response_body = urlopen('https://api-samenmeten.rivm.nl/v1.0/Things')
raw_data = response_body.read()

# creat columns
data = json.loads(raw_data)
a = "name"
station_name = jsonpath.jsonpath(data, "$..name")
station_id = jsonpath.jsonpath(data,"@iot.selfLink")

print (station_name)
#station_coordinates = jsonpath.jsonpath(data,'$..coordinates')
'''

In [3]:
# request url
response_body = urlopen('https://api-samenmeten.rivm.nl/v1.0/Things')
raw_data = response_body.read()
str_data = bytes.decode(raw_data)
json_data = json.loads(str_data)

print(type(json_data))
print(json_data.keys())

<class 'dict'>
dict_keys(['@iot.nextLink', 'value'])


In [4]:
station_id = pd.json_normalize(json_data, record_path=['value'])[['@iot.id']]
station_name = pd.json_normalize(json_data, record_path=['value'])[['name']]
station_owner = pd.json_normalize(json_data,  record_path=['value'])[['properties.owner']]
station_Datastreamslink = pd.json_normalize(json_data,  record_path=['value'])[['Datastreams@iot.navigationLink']]
print (station_id)
print (station_name)
print (station_owner)
print (station_Datastreamslink)

     @iot.id
0       6541
1       6540
2       6539
3       6538
4       6537
..       ...
195     6346
196     6345
197     6344
198     6343
199     6342

[200 rows x 1 columns]
                    name
0              LTD_59577
1              LTD_59568
2              LTD_59511
3              LTD_59473
4              LTD_59464
..                   ...
195  LUC_356726100944131
196  LUC_355523767674908
197  LUC_355523767674882
198  LUC_355523767674874
199  LUC_355523767674791

[200 rows x 1 columns]
       properties.owner
0             Luftdaten
1             Luftdaten
2             Luftdaten
3             Luftdaten
4             Luftdaten
..                  ...
195  Gemeente Rotterdam
196  Gemeente Rotterdam
197  Gemeente Rotterdam
198  Gemeente Rotterdam
199  Gemeente Rotterdam

[200 rows x 1 columns]
                        Datastreams@iot.navigationLink
0    https://api-samenmeten.rivm.nl/v1.0/Things(654...
1    https://api-samenmeten.rivm.nl/v1.0/Things(654...
2    https://api-sa

In [6]:
a = 0
b = 0
location_link = pd.json_normalize(json_data, record_path = ['value'])[['Locations@iot.navigationLink']]
station_lat2 = pd.DataFrame(columns = ['longitude'])
station_lon2 = pd.DataFrame(columns = ['latitude'])

while a < len(location_link):
    #step1：get url
    url_location = str(location_link.loc[a,'Locations@iot.navigationLink'])

    #step2：transfer url to dict
    response_location = urlopen (url_location)
    raw_location = response_location.read()
    str_location = bytes.decode(raw_location)
    json_location = json.loads(str_location)

    #step3：get target value
    station_coordinates = pd.json_normalize(json_location, record_path=['value','location','coordinates'])
    station_lat = station_coordinates.loc[0,0]
    station_lon = station_coordinates.loc[1,0]

    #step4：put the value into dataframe
    station_lat2.loc[b,'longitude'] = station_lat
    station_lon2.loc[b,'latitude'] = station_lon

    a = a + 1
    b = b + 1

    #step5：repeat
print (station_lat2)
print (station_lon2)

    longitude
0       5.406
1       5.434
2       6.206
3       5.728
4       6.122
..        ...
195     4.487
196         0
197         0
198         0
199         0

[200 rows x 1 columns]
    latitude
0     52.122
1     52.128
2     52.136
3     53.214
4     52.239
..       ...
195   51.906
196        0
197        0
198        0
199        0

[200 rows x 1 columns]


In [7]:
Joint1 = pd.concat([station_id, station_name, station_owner, station_lat2, station_lon2, station_Datastreamslink], axis=1)
Joint1.rename(columns = {'Datastreams@iot.navigationLink' : '@iot.selfLink'}, inplace = True)
Joint1.set_index('@iot.selfLink',inplace = True)
print (Joint1)

                                                    @iot.id  \
@iot.selfLink                                                 
https://api-samenmeten.rivm.nl/v1.0/Things(6541...     6541   
https://api-samenmeten.rivm.nl/v1.0/Things(6540...     6540   
https://api-samenmeten.rivm.nl/v1.0/Things(6539...     6539   
https://api-samenmeten.rivm.nl/v1.0/Things(6538...     6538   
https://api-samenmeten.rivm.nl/v1.0/Things(6537...     6537   
...                                                     ...   
https://api-samenmeten.rivm.nl/v1.0/Things(6346...     6346   
https://api-samenmeten.rivm.nl/v1.0/Things(6345...     6345   
https://api-samenmeten.rivm.nl/v1.0/Things(6344...     6344   
https://api-samenmeten.rivm.nl/v1.0/Things(6343...     6343   
https://api-samenmeten.rivm.nl/v1.0/Things(6342...     6342   

                                                                   name  \
@iot.selfLink                                                             
https://api-samenmeten.rivm.nl

In [8]:
a = 0
b = 0
station_ave_result = pd.DataFrame(columns = ['average_value'])
Joint2 = pd.DataFrame(columns = ['@iot.selfLink', 'description', 'average_value', 'unitOfMeasurement.symbol'])
datastreams_link = pd.json_normalize(json_data, record_path = ['value'])[['Datastreams@iot.navigationLink']]

while a < len (datastreams_link):
    url_datastreams = str(datastreams_link.loc[a,'Datastreams@iot.navigationLink'])
    response_datastreams = urlopen (url_datastreams)
    raw_datastreams = response_datastreams.read()
    str_datastreams = bytes.decode(raw_datastreams)
    json_datastreams = json.loads(str_datastreams)
    
    station_symbols = pd.json_normalize(json_datastreams,  record_path = ['value'])[['unitOfMeasurement.symbol']]
    station_descriptions = pd.json_normalize(json_datastreams,  record_path = ['value'])[['description']]
    station_datastream_selflinks = pd.json_normalize(json_datastreams,  record_path = ['value'])[['@iot.selfLink']]
    station_datastream_selflinks.iloc[:,0] = url_datastreams
    
#    print (datastreams_link)
#    print (station_descriptions)
#    print (station_datastream_selflinks)
    
    
    obeservations_link = pd.json_normalize(json_datastreams, record_path = ['value'])[['Observations@iot.navigationLink']]
    
    while b < len (obeservations_link):
        url_obeservations = str(obeservations_link.loc[b,'Observations@iot.navigationLink'])
        response_obeservations = urlopen (url_obeservations)
        raw_obeservations = response_obeservations.read()
        str_obeservations = bytes.decode(raw_obeservations)
        json_obeservations = json.loads(str_obeservations)
        
        try:
            station_resulttime = pd.json_normalize(json_obeservations,  record_path = ['value'])[['phenomenonTime']]
        except:
            station_ave_result.loc[b,'average_value'] =  None
        else:
            station_results = pd.json_normalize(json_obeservations,  record_path = ['value'])[['result']]
            station_ave_result.loc[b,'average_value'] = station_results['result'].mean()
            
        b = b + 1
        
    Combination = pd.concat([station_datastream_selflinks, station_descriptions, station_ave_result, station_symbols], axis=1)
    Joint2 = pd.concat([Joint2, Combination], ignore_index = True)
    
    a = a + 1
    b = 0
    station_ave_result = pd.DataFrame(columns = ['average_value'])


print (len (obeservations_link))
#print (station_resulttime)
print (Joint2)


5
                                          @iot.selfLink  \
0     https://api-samenmeten.rivm.nl/v1.0/Things(654...   
1     https://api-samenmeten.rivm.nl/v1.0/Things(654...   
2     https://api-samenmeten.rivm.nl/v1.0/Things(654...   
3     https://api-samenmeten.rivm.nl/v1.0/Things(654...   
4     https://api-samenmeten.rivm.nl/v1.0/Things(654...   
...                                                 ...   
1071  https://api-samenmeten.rivm.nl/v1.0/Things(634...   
1072  https://api-samenmeten.rivm.nl/v1.0/Things(634...   
1073  https://api-samenmeten.rivm.nl/v1.0/Things(634...   
1074  https://api-samenmeten.rivm.nl/v1.0/Things(634...   
1075  https://api-samenmeten.rivm.nl/v1.0/Things(634...   

                          description average_value unitOfMeasurement.symbol  
0                    LTD_59577-6-pres          None                      hPa  
1                      LTD_59577-6-rh          None                        %  
2                    LTD_59577-6-temp          None 

In [11]:
result = Joint2.join(Joint1, on='@iot.selfLink')
print (result)
result.to_excel('open_air_pollution.xlsx')

                                          @iot.selfLink  \
0     https://api-samenmeten.rivm.nl/v1.0/Things(654...   
1     https://api-samenmeten.rivm.nl/v1.0/Things(654...   
2     https://api-samenmeten.rivm.nl/v1.0/Things(654...   
3     https://api-samenmeten.rivm.nl/v1.0/Things(654...   
4     https://api-samenmeten.rivm.nl/v1.0/Things(654...   
...                                                 ...   
1071  https://api-samenmeten.rivm.nl/v1.0/Things(634...   
1072  https://api-samenmeten.rivm.nl/v1.0/Things(634...   
1073  https://api-samenmeten.rivm.nl/v1.0/Things(634...   
1074  https://api-samenmeten.rivm.nl/v1.0/Things(634...   
1075  https://api-samenmeten.rivm.nl/v1.0/Things(634...   

                          description average_value unitOfMeasurement.symbol  \
0                    LTD_59577-6-pres          None                      hPa   
1                      LTD_59577-6-rh          None                        %   
2                    LTD_59577-6-temp          None